In [11]:
import requests
import re
import csv

In [12]:
import lxml.html as lh

In [13]:
from bs4 import BeautifulSoup

In [14]:
r=requests.get("https://www.mscorecard.com/mscorecard/courses.php?page=1&CourseName=&Country=USA&SubmitButton=Search", headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})

In [15]:
c=r.content
soup=BeautifulSoup(c,"html.parser")

In [20]:
pre_url="https://www.mscorecard.com/mscorecard/courses.php?page="
post_url="&CourseName=&Country=USA&SubmitButton=Search"

In [21]:
page_nr=soup.find_all('select', {"class":"form-control"}) [1]
options = page_nr.find_all("option")
last_page=[y.text for y in options][-1]
first_page=[y.text for y in options][0]
header={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'}
#values = [o.get("value") for o in options]
#print(last_page)
#print (first_page)
#print(values)
#print(page_nr)
file = open('GC_course.csv', 'w')
writer = csv.writer(file)

IndexError: list index out of range

In [22]:
# write title row
writer.writerow(['GC Name', 'Address', 'City', 'State', 'Zip', 'Phone', 'Email', 'Website'])

52

In [23]:
for page in range(int(first_page),int(last_page)+1):
    #print(pre_url+str(page)+post_url)
    gc=(pre_url+str(page)+post_url)
    #print(gc)
    r1=requests.get(gc)
    c1=r1.content
    soup=BeautifulSoup(c1,"html.parser")
    #print(soup.prettify())
    all_gc = [tag['href'] for tag in soup.find_all('a', {'class':"no-hover"})]
    #print(all_gc)
    l=[]
    for item in all_gc:
            r2=requests.get("https://www.mscorecard.com/mscorecard/"+str(item))
            c2=r2.content
            soup=BeautifulSoup(c2,"html.parser")
            #print(soup.prettify())
            course = soup.find('div', class_='page-content')
            gc_name = course.h1.text
            addr = soup.find('div', class_='col-md-4')
            innerHtml = addr.decode_contents()
            addr_list =innerHtml.split('<br/>')
            x=re.findall(r'\d+',addr_list[1])
            str1=""
            for ele in x:
                str1 += ele
       
            #print (addr_list[1].replace(str1,'').split(',')[1])
            web = soup.find('div', class_='col-md-8')
            #print(gc_name)
            #print(addr.text.replace("USA",""))
            #print(web.a.text)
            #print(web.div.text.replace("phone",""))
            #print(web.select('[href^=mailto]'))
            #print(web.select('[href^=http]'))
            #print([a.text for a in web.select("a[href^=mailto:]")])
            
            ### start here
            gc_addr=addr_list[0]
            gc_city=addr_list[1].replace(str1,'').split(',')[0]
            try:
                gc_state=addr_list[1].replace(str1,'').split(',')[1]
            except:
                gc_state=''
            try:
                gc_zip=str1
            except:
                gc_zip=''
            try:
                gc_phone=web.div.text.replace("phone","")
            except:
                gc_phone=''
            try:
                gc_email=web.select('a[href^=mailto]')
            except:
                gc_email=''
            try:
                gc_web=web.select('a[href^=http]')
            except:
                gc_web=''
                
            print (str(gc_name) + ' ' + str(gc_addr) + ' ' + str(gc_city) + ' ' + str(gc_state) + ' ' + str(gc_zip) + ' ' + str(gc_phone) + ' ' + str(gc_email) + ' ' + str(gc_web))            

            #writer.writerow([gc_name.encode('utf-8'), gc_addr.encode('utf-8'), gc_city.encode('utf-8'), gc_state.encode('utf-8'), gc_zip.encode('utf-8'), gc_phone.encode('utf-8'), gc_email.encode('utf-8'), gc_web.encode('utf-8')])
            writer.writerow([gc_name, gc_addr, gc_city, gc_state, gc_zip, gc_phone, gc_email, gc_web])
   

1757 Golf Club 45120 Waxpool Road  Dulles  VA 20166 +1 703.444.0901 [] [<a href="http://www.1757golfclub.com" target="_blank">http://www.1757golfclub.com</a>]
3 Creek Ranch 2800 Ranch House Cir  Jackson Hole  WY 83001 (800) 920-9292 [<a href="mailto:mconnaughton@3creekranch-jh.com">mconnaughton@3creekranch-jh.com</a>] [<a href="http://www.3creekranch-jh.com" target="_blank">http://www.3creekranch-jh.com</a>]
3 Lakes Golf Course 6700 Saltsburg Rd  Pittsburgh  PA 15235 412-793-7111 [<a href="mailto:golfshop@3lakesgolf.com">golfshop@3lakesgolf.com</a>] [<a href="http://www.3lakesgolf.com/" target="_blank">http://www.3lakesgolf.com/</a>]
3 Lakes Golf Course 6700 Saltsburg Rd  Pittsburgh  PA 15235 412-793-7111 [<a href="mailto:golfshop@3lakesgolf.com">golfshop@3lakesgolf.com</a>] [<a href="http://www.3lakesgolf.com/" target="_blank">http://www.3lakesgolf.com/</a>]
3/30 Club 101 Country Club Ln  Lowden  IA 52255 (319) 944-7695 [<a href="mailto:katimmer22@hotmail.com">katimmer22@hotmail.com</

KeyboardInterrupt: 

In [9]:
file.close()